# Charon4 txt to Python, 2018 data

# different export settings compared to the recent data!
The following screenshot shows the expected settings while exporting the 2018 data. They are exported per day, for easier reading of the different cable lengths.

<img src="./pictures/Charon4_export_window_txt_2018_day.png" alt="drawing" width="1300"/>

In [2]:
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt
from datetime import date, timedelta
from collections import defaultdict
import pickle

from my_func_mvw.functions import get_abspath, read_pickle

In [4]:
##############Input##############################
path_DTS_processed = r"..\Alsdorf\Daten\DTS_processed"
data_save_csv    =True # True False
data_save_pickle =True # True False
##################################################

In [5]:
controller=3188 # 3188: Alsdorf
# generate all requested paths, some paths may be empty
path_to_controller_2018_day=path_DTS_processed + r"\..\DTS_unprocessed\Charon4\charon4_export_as_txt\2018_export_by_day"
year="2018"
channels=[1,2,3,4,5,6,7,8]
months=[1,2,3,4,5,6,7,8,9,10,11,12]
paths_for_activate_2018_day={}

for channel in channels:
    for month in months:
        paths_for_activate_2018_day[f"temp_ch{channel}_year2018_month{month}"] = path_to_controller_2018_day + f"\Controller\{controller}\{channel}\Temperature Data\{year}\{month}"
#paths_for_activate_2018_day

In [6]:
def import_temp_to_df_2018(path):
    """"""
    one_file = pd.read_csv(path,decimal=".",delimiter="\t",skiprows=7,index_col=0)
    one_file = one_file.drop(one_file.columns[0:2],axis=1)
    one_file.index = pd.to_datetime(one_file.index, infer_datetime_format=True)
    one_file.columns = one_file.columns.astype(float)
    one_file.index.names = ['Date']
    one_file.columns.names=["Length [m]"]
    return one_file

def create_base_dataframe(n_columns,sampling_05=False):
    """"""
    if sampling_05==False: #sampling interval is 1 m
        df=pd.DataFrame(columns=np.linspace(0,n_columns-1,n_columns))
        df.rename(columns=lambda x: float(x), inplace=True) #to have all 2018 data data as float
    elif sampling_05==True:
        df=pd.DataFrame(columns=np.linspace(0,(n_columns-1)/2,n_columns))
    df.index.names = ['Date']
    df.columns.names=["Length [m]"]
    return df

In [7]:
# Read all files exported by Charon

# Create base dic: depending on channel number and cable length
cable_lengths_2018=[233,1314,2340,1171,1170,798,259,799,268,1352,676,223,378,250,800]
# data_2018_day, is named day because the data is exported different from Charon4, than the data from the other years
data_2018_day=my_dict = defaultdict(lambda: defaultdict(lambda: defaultdict(dict))) #three level defaultdict
for channel in channels:
    for cable_type in cable_lengths_2018:
        for month in months:

            if cable_type in [233,1314,1171,1170,798,259,799,268,676,223,378,250,800]: # sampling interval is 1 m
                data_2018_day[cable_type][str(channel)][str(month)] = create_base_dataframe(cable_type)

            elif cable_type in [2340, 1352]: #sampling interval is 0.5 m instead of 1 m
                data_2018_day[cable_type][str(channel)][str(month)] = create_base_dataframe(cable_type,sampling_05=True)

# Read the data
count_ParserError=0
for channel in channels:
    for month in months: #path could be empty --> no data 
        my_path=paths_for_activate_2018_day[f"temp_ch{channel}_year2018_month{month}"]
        all_my_paths=get_abspath(my_path + "\*\*") 

        for my_file in all_my_paths: # check cable length and create different dataframes
            try: # Read Data
                one_file = import_temp_to_df_2018(my_file)

                #check cable length and put it in corresponding dataframe
                n_columns=len(one_file.columns)
                data_2018_day[n_columns][str(channel)][str(month)]=pd.concat([data_2018_day[n_columns][str(channel)][str(month)],one_file],axis=0)
                # doppelte datums tauchen auf

            except pd.errors.ParserError: # except ParserError, file contains data with different cable lengths
                # With this solution I will skip these data.
                count_ParserError+=1
                pass
        
        # check if one folder contains more than one file
        check_for_multiple_files=True
        if check_for_multiple_files:
            path_to_day_folders=get_abspath(my_path + "\*")
            for day_folder in path_to_day_folders:
                n_files_in_dayfolder=len(get_abspath(day_folder + "\*"))
                if n_files_in_dayfolder != 1:
                    print("Warning: a day folder conatins mpre than one file")

print(f"{count_ParserError} ParserErrors were skipped. This means {count_ParserError} days with data are not read!")        

2 ParserErrors were skipped. This means 2 days with data are not read!


In [8]:
def save_helper(data_2018_day,channel,cable_length,create_base_dataframe=create_base_dataframe):
    """merges different month for each channel into one dataframe"""
    if cable_length in [233,1314,1171,1170,798,259,799,268,676,223,378,250,800]: # sampling interval is 1 m
        data_2018_day_save = create_base_dataframe(cable_length)
    elif cable_length in [2340, 1352]: #sampling interval is 0.5 m instead of 1 m
        data_2018_day_save = create_base_dataframe(cable_length,sampling_05=True)

    for month in data_2018_day[cable_length][channel].keys():
        one_file = data_2018_day[cable_length][channel][month]
        data_2018_day_save = pd.concat([data_2018_day_save , one_file],axis=0) 

    return data_2018_day_save

In [9]:
# Save Data 2018 csv
if data_save_csv:
    save_to_path = path_DTS_processed + r"\temp_2018_Controller3188\csv"
    for cable_length in data_2018_day.keys():
        for channel in data_2018_day[cable_length].keys():
            data_2018_day_save = save_helper(data_2018_day,channel,cable_length)

            # Save data
            if data_2018_day_save.shape[0]!=0: # dataframe contains rows (with data)
                filename=f"\\temp_cablelength{cable_length}_ch{channel}.csv"
                data_2018_day_save.to_csv(save_to_path + filename)

In [10]:
# Save Data 2018 pickle
if data_save_pickle:
    def write_pickle(save_to:str,data_2018_day):
        #Function to write pickle Files
        for cable_length in data_2018_day.keys():
            for channel in data_2018_day[cable_length].keys():
                data_2018_day_save = save_helper(data_2018_day,channel,cable_length)

                #if len(data_2018_day[cable_length][channel].keys()) != 0:
                if data_2018_day_save.shape[0]!=0: # dataframe contains rows (with data)
                    filename=f"\\temp_cablelength{cable_length}_ch{channel}"
                    with open(save_to + filename, 'wb') as handle:
                        pickle.dump(data_2018_day_save, handle, protocol=pickle.HIGHEST_PROTOCOL)

    save_to_path_pickle = path_DTS_processed + r"\temp_2018_Controller3188\pickle"
    write_pickle(save_to_path_pickle,data_2018_day)

    # Load pickle data - test
    filename=r"\temp_cablelength378_ch5"
    load_file_pickle = save_to_path_pickle + filename
    test_load_pickle=read_pickle(load_file_pickle)
    #test_load_pickle

In [ ]:
# To-Do - not important
# TRT data from sciebo: sciebo\DTS Data\Alsdorf\Nullmessung_und_TRT_2018
# check if equal to the data here


# später weiter machen

In [13]:
trt_data_sciebo={}

path = r"..\Alsdorf\Nullmessung_und_TRT_2018\Natural State" + "\CH_2"



path = r"..\Alsdorf\Nullmessung_und_TRT_2018\TRT" #+ "\CH_1"
path_to_files = get_abspath(path + "\CH_2\*")
for filename in path_to_files:
    trt_data_sciebo[filename] = pd.read_csv(filename,delimiter = ',',index_col=0, header=7 )
    trt_data_sciebo[filename].index = pd.to_datetime(trt_data_sciebo[filename].index, dayfirst = True).tz_localize(None)
    trt_data_sciebo[filename] = trt_data_sciebo[filename].drop(trt_data_sciebo[filename].columns[[0,1]], axis=1)

trt_data_sciebo[path_to_files[0]]

,0.00,1.00,2.00,3.00,4.00,5.00,6.00,7.00,8.00,9.00,...,666.00,667.00,668.00,669.00,670.00,671.00,672.00,673.00,674.00,675.00
,,,,,,,,,,,,,,,,,,,,,
2018-07-09 14:01:21,27.73,27.27,26.43,26.21,26.50,26.53,26.34,26.19,25.98,25.55,...,23.09,23.09,23.16,23.11,23.04,22.92,22.78,22.89,23.04,23.00
2018-07-09 14:09:21,27.53,27.05,26.36,26.24,26.35,26.28,26.20,26.20,26.08,25.72,...,23.22,23.14,23.14,23.18,23.15,22.98,22.85,22.98,23.13,23.13
2018-07-09 14:17:22,27.61,27.02,26.33,26.19,26.28,26.24,26.25,26.40,26.31,25.83,...,23.40,23.18,23.18,23.35,23.33,23.03,22.81,22.97,23.28,23.45
2018-07-09 14:25:22,27.78,27.32,26.62,26.34,26.47,26.57,26.59,26.65,26.50,25.98,...,23.49,23.42,23.38,23.38,23.32,23.03,22.83,23.08,23.46,23.67
2018-07-09 14:33:22,28.19,27.77,26.96,26.58,26.75,26.90,26.89,26.89,26.67,26.09,...,23.65,23.59,23.39,23.20,23.15,23.05,22.96,23.15,23.38,23.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-07-10 15:14:28,21.34,20.92,20.38,20.40,20.58,20.66,20.72,20.62,20.48,20.36,...,23.60,23.61,23.66,23.71,23.70,23.59,23.47,23.54,23.66,23.72
2018-07-10 15:22:28,21.35,20.84,20.35,20.46,20.61,20.57,20.54,20.54,20.53,20.37,...,23.63,23.54,23.50,23.58,23.74,23.64,23.38,23.44,23.60,23.62
2018-07-10 15:30:29,21.36,20.88,20.38,20.50,20.66,20.58,20.51,20.46,20.51,20.45,...,23.50,23.51,23.44,23.42,23.53,23.48,23.34,23.45,23.55,23.56


In [19]:

data_in_database=read_pickle(save_to_path_pickle + r"\temp_cablelength676_ch2")
data_in_database.index[10]

Timestamp('2018-07-10 01:21:51')

In [26]:
# for loc in trt_data_sciebo[path_to_files[0]].index:

#später weiter machen

,0.00,1.00,2.00,3.00,4.00,5.00,6.00,7.00,8.00,9.00,...,666.00,667.00,668.00,669.00,670.00,671.00,672.00,673.00,674.00,675.00
,,,,,,,,,,,,,,,,,,,,,
2018-07-09 14:01:21,27.73,27.27,26.43,26.21,26.50,26.53,26.34,26.19,25.98,25.55,...,23.09,23.09,23.16,23.11,23.04,22.92,22.78,22.89,23.04,23.00
2018-07-09 14:09:21,27.53,27.05,26.36,26.24,26.35,26.28,26.20,26.20,26.08,25.72,...,23.22,23.14,23.14,23.18,23.15,22.98,22.85,22.98,23.13,23.13
2018-07-09 14:17:22,27.61,27.02,26.33,26.19,26.28,26.24,26.25,26.40,26.31,25.83,...,23.40,23.18,23.18,23.35,23.33,23.03,22.81,22.97,23.28,23.45
2018-07-09 14:25:22,27.78,27.32,26.62,26.34,26.47,26.57,26.59,26.65,26.50,25.98,...,23.49,23.42,23.38,23.38,23.32,23.03,22.83,23.08,23.46,23.67
2018-07-09 14:33:22,28.19,27.77,26.96,26.58,26.75,26.90,26.89,26.89,26.67,26.09,...,23.65,23.59,23.39,23.20,23.15,23.05,22.96,23.15,23.38,23.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-07-10 15:14:28,21.34,20.92,20.38,20.40,20.58,20.66,20.72,20.62,20.48,20.36,...,23.60,23.61,23.66,23.71,23.70,23.59,23.47,23.54,23.66,23.72
2018-07-10 15:22:28,21.35,20.84,20.35,20.46,20.61,20.57,20.54,20.54,20.53,20.37,...,23.63,23.54,23.50,23.58,23.74,23.64,23.38,23.44,23.60,23.62
2018-07-10 15:30:29,21.36,20.88,20.38,20.50,20.66,20.58,20.51,20.46,20.51,20.45,...,23.50,23.51,23.44,23.42,23.53,23.48,23.34,23.45,23.55,23.56
